In [3]:
# Necessary Imports
import pandas as pd
import numpy as np

In [4]:
# Load Data
def load_data(csv_file):
    names = ["question", "type"]
    dataset_all = pd.read_csv(csv_file,names=names)
    return dataset_all

dataset_all = load_data('nikiai_train.csv')

In [13]:
# Peek into the Data
dataset_all.head(5)

,question,type
0,how did serfdom develop in and then leave russ...,unknown
1,what films featured the character popeye doyle ?,what
2,how can i find a list of celebrities ' real na...,unknown
3,what fowl grabs the spotlight after the chines...,what
4,what is the full form of .com ?,what


In [16]:
print("Shape - {}").format(dataset_all.shape)

Shape - (1483, 2)


In [21]:
# Preapre data - get features and labels
def preapre_data(dataset_all):
    dataset=dataset_all.values
    X_Train = dataset[:,0]
    Y_Train = dataset[:,1]
    return  X_Train, Y_Train

X_Train, Y_Train = preapre_data(dataset_all)

In [22]:
print X_Train[:10]
print Y_Train[:10]

['how did serfdom develop in and then leave russia ? '
 'what films featured the character popeye doyle ? '
 "how can i find a list of celebrities ' real names ? "
 'what fowl grabs the spotlight after the chinese year of the monkey ? '
 'what is the full form of .com ? '
 'what contemptible scoundrel stole the cork from my lunch ? '
 "what team did baseball 's st. louis browns become ? "
 'what is the oldest profession ? ' 'what are liver enzymes ? '
 'name the scar-faced bounty hunter of the old west . ']
['unknown' 'what' 'unknown' 'what' 'what' 'what' 'what' 'what' 'what'
 'unknown']


In [38]:
# Clean Data
def remove_questionmark(slist):
    new_x = []
    for x in slist:
         new_x.append(x.replace("?",""))
    return new_x
    
X_Train = remove_questionmark(X_Train)
X_Train = map(str.rstrip,X_Train)

from string import digits

def remove_numbers(slist):
    res = map(lambda x: x.translate(None, digits), slist)
    return res

X_Train = remove_numbers(X_Train)

In [49]:
# Encode labels
from sklearn.preprocessing import LabelEncoder

def encode_lables(Y_Train):
    encoder = LabelEncoder()
    encoder.fit(Y_Train)
    encoded_y = encoder.transform(Y_Train)
    return  encoder, encoded_y

encoder, encoded_y = encode_lables(Y_Train)

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

def vectorise_featres(X_Train):
    vectorizer = CountVectorizer(analyzer="word", preprocessor=None, tokenizer=None, stop_words=None, max_features=5000)
    train_data_features = vectorizer.fit_transform(X_Train)
    # Numpy arrays are easy to work with, so convert the result to an array
    train_data_features = train_data_features.toarray()
    return vectorizer, train_data_features

vectorizer, train_data_features = vectorise_featres(X_Train)

In [42]:
# Exploring the vectorised features
print train_data_features[0]
print('The dimension of train_data_features is {}.'.format(train_data_features.shape))

[0 0 0 ..., 0 0 0]
The dimension of train_data_features is (1483, 3525).


In [43]:
vocab = vectorizer.get_feature_names()
print(vocab)

[u'abbie', u'abbreviated', u'abbreviation', u'abby', u'abilities', u'abolished', u'abominable', u'about', u'academy', u'acceptance', u'access', u'accessory', u'accident', u'accidents', u'accommodate', u'accompanied', u'accompanying', u'according', u'account', u'accurate', u'accurately', u'acetylsalicylic', u'acid', u'acreage', u'across', u'act', u'acted', u'actor', u'actress', u'actual', u'ad', u'adam', u'add', u'adding', u'address', u'addresses', u'adenauer', u'administration', u'admiral', u'admit', u'admitted', u'admonition', u'ads', u'adult', u'advanced', u'adventures', u'adventuring', u'advertise', u'advertised', u'advertizing', u'advised', u'aerodynamics', u'aeul', u'affairs', u'affectionate', u'afghanistan', u'africa', u'after', u'against', u'age', u'aged', u'agency', u'ago', u'agree', u'ahead', u'aids', u'aim', u'air', u'airborne', u'airplane', u'airport', u'airports', u'airwaves', u'akita', u'al', u'alaska', u'albert', u'album', u'aldous', u'aldrin', u'ali', u'alice', u'alive',

In [51]:
# Preapre, Preprocess Test Data
def preapare_test_data(X_Test):
    X_Test= map(str.lower,X_Test)
    X_Test = remove_questionmark(X_Test)
    X_Test = map(str.rstrip,X_Test)
    X_Test = remove_numbers(X_Test)

    #Test data preapre
    #vectorizer = CountVectorizer(analyzer="word", preprocessor=None, tokenizer=None, stop_words=None, max_features=5000)
    test_data_features = vectorizer.transform(X_Test)
    # Numpy arrays are easy to work with, so convert the result to an array
    test_data_features = test_data_features.toarray()
    return test_data_features

test_data_features = preapare_test_data(X_Test)
    #print('The dimension of test_data_features is {}.'.format(test_data_features.shape))

In [52]:
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier

def run_rfClassifier():
    rf_clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0) 

    # Cross validation to evaluate the performance of Random Forest
    rf_clf_error = cross_val_score(rf_clf, train_data_features, encoded_y, 
                                       cv=5, scoring='accuracy', n_jobs=-1).mean()

    print('Random Forest Accuracy: {:.4}'.format(rf_clf_error*100))
    return rf_clf

rf_clf = run_rfClassifier()

Random Forest Accuracy: 95.48


**Make Predictions**

In [50]:
rf_clf.fit(train_data_features, encoded_y)

# Make predictions
y = rf_clf.predict(test_data_features)
encoder.inverse_transform(y)

array(['unknown', 'who', 'what', 'unknown', 'affirmation', 'what', 'when',
       'affirmation', 'when', 'when'], dtype=object)